In [1]:
import cv2

In [2]:
import os
import glob
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = 224
AUGMENT_TIMES = 5  # Nhân mỗi ảnh lên 5 lần

# Khởi tạo bộ sinh ảnh
augmentor = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

def augment_and_save_images(class_name):
    input_path = os.path.join('DataTranning', class_name)
    image_paths = glob.glob(os.path.join(input_path, '*'))

    for img_path in image_paths:
        img = cv2.imread(img_path)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img / 255.0
        img = np.expand_dims(img, axis=0)

        # Tạo ảnh mới
        aug_iter = augmentor.flow(img, batch_size=1)
        base_name = os.path.splitext(os.path.basename(img_path))[0]

        for i in range(AUGMENT_TIMES):
            aug_img = next(aug_iter)[0] * 255.0
            aug_img = aug_img.astype(np.uint8)
            aug_img = cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR)

            save_path = os.path.join(input_path, f"{base_name}_aug{i}.jpg")
            cv2.imwrite(save_path, aug_img)

# Thực hiện nhân ảnh cho từng lớp
augment_and_save_images('good')
augment_and_save_images('error')


In [3]:
import os
import shutil
import random

def split_dataset(source_dir, target_dir, split_ratio=0.2):
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    class_names = os.listdir(source_dir)
    for class_name in class_names:
        src_class_dir = os.path.join(source_dir, class_name)
        dst_class_dir = os.path.join(target_dir, class_name)

        if not os.path.exists(dst_class_dir):
            os.makedirs(dst_class_dir)

        # Lấy danh sách ảnh
        image_files = [f for f in os.listdir(src_class_dir) if os.path.isfile(os.path.join(src_class_dir, f))]

        # Xáo trộn và chọn ảnh để copy
        random.shuffle(image_files)
        num_to_copy = int(len(image_files) * split_ratio)
        images_to_copy = image_files[:num_to_copy]

        for img_name in images_to_copy:
            src_img_path = os.path.join(src_class_dir, img_name)
            dst_img_path = os.path.join(dst_class_dir, img_name)
            shutil.copy2(src_img_path, dst_img_path)

# Thực hiện chia ảnh từ DataTranning sang DataTest
split_dataset('DataTranning', 'DataTest', split_ratio=0.2)
